In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

2023-06-04 17:52:42.324497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 17:52:43.014487: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
samples = []
IMAGES_DIR="images/esp32/good/"
for dir,_,files in os.walk(IMAGES_DIR):
    for file in files:
        samples.append(np.asarray(Image.open(os.path.join(dir, file)), dtype=np.float32).reshape(29, 40, 1))
sample_array = np.asarray(samples)
sample_array.shape

(638, 29, 40, 1)

In [3]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(sample_array).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

In [4]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('./output')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

2023-06-04 17:52:44.068157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 17:52:44.086113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 17:52:44.086156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 17:52:44.089216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-04 17:52:44.089264: I tensorflow/compile